In [ ]:
import kagglehub
kazanova_sentiment140_path = kagglehub.dataset_download('kazanova/sentiment140')

print('Data source import complete.')


100%|██████████| 80.9M/80.9M [00:01<00:00, 58.1MB/s]

Extracting files...


Data source import complete.


In [ ]:
import os

file_path = os.path.join(kazanova_sentiment140_path, 'training.1600000.processed.noemoticon.csv')

## Importing the dataset

In [ ]:
df=pd.read_csv(file_path,encoding='ISO-8859-1',header=None)
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


## Feature Engineering

In [ ]:
columns=df.columns
columns

Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [ ]:
df.drop([1,2,3,4],axis=1,inplace=True)
df.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df.columns=['sentiment','data']
df.head()

,sentiment,data
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
y=df['sentiment']

## Splitting the dataset in train and test split

In [ ]:
from sklearn.model_selection import train_test_split
df_train,df_test,y_train,y_test=train_test_split(df['data'],y,test_size=0.33,random_state=42)
print('DF Train Shape: ',df_train.shape)
print('DF Test Shape: ',df_test.shape)
print('Y Train Shape: ',y_train.shape)
print('Y Test Shape: ',y_test.shape)


DF Train Shape:  (1072000,)
DF Test Shape:  (528000,)
Y Train Shape:  (1072000,)
Y Test Shape:  (528000,)


## Building deep learn model

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
max_words=1000
tokenizer=Tokenizer(max_words)
tokenizer.fit_on_texts(df_train)
sequence_train=tokenizer.texts_to_sequences(df_train)
sequence_test=tokenizer.texts_to_sequences(df_test)

In [ ]:
word2vec=tokenizer.word_index
V=len(word2vec)
print('dataset has %s number of independent tokens' %V)

dataset has 527470 number of independent tokens


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
data_train=pad_sequences(sequence_train)
data_train.shape

(1072000, 42)

In [ ]:
T=data_train.shape[1]
data_test=pad_sequences(sequence_test,maxlen=T)
data_test.shape

(528000, 42)

In [ ]:
from tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding, Dropout,Flatten
from tensorflow.keras.models import Model, Sequential

In [ ]:
#change the 4 to 1
y_train=np.array(y_train)
y_test=np.array(y_test)

In [ ]:
# prompt: in y_train and y_test, change the values from 4 to 1

y_train[y_train == 4] = 1
y_test[y_test == 4] = 1

In [ ]:
y_train




array([0, 1, 1, ..., 0, 0, 0])

In [ ]:
cnn_model = Sequential()
cnn_model.add(Embedding(1000,50))
cnn_model.add(Dropout(.5))
cnn_model.add(Conv1D(32,3,padding = 'valid', activation = 'relu'))
cnn_model.add(MaxPooling1D())
cnn_model.add(Conv1D(32,3, padding = 'valid', activation = 'relu'))
cnn_model.add(MaxPooling1D())
cnn_model.add(Flatten())
cnn_model.add(Dense(250, activation ='relu'))
cnn_model.add(Dropout(.5))
cnn_model.add(Dense(1, activation='sigmoid'))
cnn_model.summary()


In [ ]:
# D=50
# i=Input((T,))
# x=Embedding(V+1,D)(i)
# x=Conv1D(32,3,activation='relu')(x)
# x=MaxPooling1D(3)(x)
# x=Conv1D(64,3,activation='relu')(x)
# x=MaxPooling1D(3)(x)
# x=Conv1D(128,3,activation='relu')(x)
# x=GlobalMaxPooling1D()(x)
# x=Dense(5,activation='softmax')(x)
# model=Model(i,x)
# model.summary()

## Training the model

In [ ]:
cnn_model.compile(loss='binary_crossentropy' ,optimizer='adam',metrics=['accuracy'])
cnn_senti=cnn_model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=10,batch_size=64)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (100, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


 2462/10720 ━━━━━━━━━━━━━━━━━━━━ 2:31 18ms/step - accuracy: 0.4975 - loss: 0.5471

KeyboardInterrupt: 

In [ ]:
plt.plot(cnn_senti.history['accuracy'])
plt.plot(cnn_senti.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(cnn_senti.history['loss'])
plt.plot(cnn_senti.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()